In [ ]:
!unzip amazon-fine-food-reviews.zip

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

## Business Case

The web contains enormous amounts of user generated data. One huge source of such information can be found in the myriads of customer written complaints and reviews.

The aim of this data challenge is to show that it is possible to extract useful insights from text and that there is value in text data. Can you convince the C-suite of an online food seller company that there is value in working with the text data? In a first meeting you want to show them, that a machine learning model can learn from the review summary and the review itself to distinguish between different levels of consumer satisfaction.



The situation in a little more detail

The food retailer wants to understand their users as well as users feelings about the products they are selling.

How could a model as described above help in this situation? Make assumptions or educated guesses if some details are missing that you would like to use for your argument.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df_reviews = pd.read_csv('Reviews.csv')

## Data Description

This dataset consists of reviews of fine foods from amazon. The data span a period of more than 10 years, including all ~500,000 reviews up to October 2012. Reviews include product and user information, ratings, and a plain text review. It also includes reviews from all other Amazon categories.

Data includes:

- Reviews from Oct 1999 - Oct 2012
- 568,454 reviews
- 256,059 users
- 74,258 products
- 260 users with > 50 reviews

Columns:

- Id Row Id
- ProductId Unique identifier for the product
- UserId Unqiue identifier for the user
- ProfileName Profile name of the user
- HelpfulnessNumerator Number of users who found the review helpful
- HelpfulnessDenominator Number of users who indicated whether they found the review helpful or not
- Score Rating between 1 and 5
- Time Timestamp for the review
- Summary Brief summary of the review
- Text Text of the review

In [ ]:
df_reviews.info()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

=> **'ProfileName' and 'Summary' contains nulls**

In [ ]:
df_reviews.describe(include='all')

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
df_reviews.head()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

### Score

In [ ]:
df_reviews['Score'].unique()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
pd.crosstab(df_reviews['Score'],'count')

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

categorical variable 'score' looks like the perfect candidate for the target variable for the sentiment analysis. It has 5 different categories ranging from 1 to 5. Let's consider the categories 0 and 1 as negative reviews, the category 2 as neutral and the categories 4 and 5 as positive reviews. But the categories are **not balanced** (category 5 has a lot more entries than category 2).

### ProductId

In [ ]:
len(df_reviews['ProductId'].unique()) 

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

There are more than one review about the same product but which product was reviewed most often?

In [ ]:
df_reviews['ProductId'].value_counts()[:10]

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

How many products were reviewed only once?

In [ ]:
df_reviews['ProductId'].value_counts()[df_reviews['ProductId'].value_counts() == 1].count()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
df_reviews['ProductId'].value_counts().value_counts()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

Many products have only a small number of reviews

### UserId

In [ ]:
df_reviews['UserId'].value_counts()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

Some users reviewed more than once, especially user 'A3OXHLG6DIBRW8' reviewed 448 times. Which products did this user review?

In [ ]:
df_reviews[df_reviews['UserId'] == df_reviews['UserId'].value_counts().index[0]]['ProductId'].value_counts()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

This user reviewed some products more than once!

How many users reviewed the same product more than once?

In [ ]:
g = df_reviews.groupby(['UserId','ProductId']).size().reset_index(name='Size')
len(g[g['Size'] > 1]['UserId'].unique())

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

1243 user reviewed a product more than once

### ProfileName

If we compare the statistics from above 'ProfileName' and 'UserId' are not always the same, i.e. the same user can have several profile names. 

Or can one profile name have more than one user id?

In [ ]:
g = df_reviews.groupby(['UserId'])['ProfileName'].agg(['nunique','unique']).reset_index()
g[g['nunique'] > 1]

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
g[g['nunique'] > 1].shape

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
df_reviews[df_reviews['UserId'] == 'AX148M3X6EUGN']

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
df_reviews[df_reviews['ProfileName'].isna()].shape

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
profile_names_nulls = df_reviews[df_reviews['ProfileName'].isna()]['UserId'].to_list()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
df_reviews[df_reviews['UserId'].isin(profile_names_nulls)]

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
df_reviews[df_reviews['UserId'].isin(profile_names_nulls)].shape

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

The profile names with nulls have no other user id's with no nulls in the profile names

In [ ]:
g = df_reviews.groupby(['ProfileName'])['UserId'].agg(['nunique','unique']).reset_index()
g[g['nunique'] > 1]

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [ ]:
g['nunique'].max()

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

We have 50 UserId's with more than one ProfileName but 10107 ProfileName's with more than one UserId. For instance 166 user id's have the same profile name.

### HelpfullnessNumerator

In [ ]:
df_reviews.boxplot(column='HelpfulnessNumerator')

Error: Jupyter cannot be started. Error attempting to locate jupyter: 